In [9]:
""" Import dependencies """
import os
import rasterio
import matplotlib
import numpy as np
import pandas as pd
from sklearn import metrics
from ML.ml_utils import ML_utils, CustomLoss
from matplotlib import pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"]="1" # Second GPU on Aulus 4
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
import keras
%matplotlib inline
matplotlib.use('agg')
plt.ioff()

In [10]:
""" Some useful methods """
def tile_xy(tile_name):
    pos = tile_name.split('_')[1].split('.')[0]
    x = int(pos.split('-')[0])
    y = int(pos.split('-')[1])
    return x,y

def fetch_sorted_tiles(path_to_tiles):
    tile_names = [tile for tile in os.listdir(path_to_tiles) if tile.endswith(".tif")]
    tile_vals_y, tile_vals_x = [], []
    for tile in tile_names:
        x,y = tile_xy(tile)
        tile_vals_x.append(x)
        tile_vals_y.append(y)
    max_val_y = max(tile_vals_y)
    max_val_x = max(tile_vals_x)
    tile_pixel_values = [*range(0,max(max_val_x,max_val_y) + 1,256)]
    pixel_index_map = {}
    for idx,pixel in enumerate(tile_pixel_values):
        pixel_index_map[pixel] = idx
    tiles = np.empty((int(max_val_x / 256)+1, int(max_val_y / 256)+1), dtype=object)
    for tile in tile_names:
        x,y = tile_xy(tile)
        x = pixel_index_map[x]
        y = pixel_index_map[y]
        tiles[x][y] = tile      
    return tiles



In [11]:
""" User defined parameters """
dataset_name = 'data_flood_test'
test_on_tiles = True
masks = False
path_to_model = '/localhome/studenter/mikaellv/Project/ML/models/DeepLabV3_Xception'

model_name = os.path.split(path_to_model)[1]
save_path_image_predictions = f'/localhome/studenter/mikaellv/Project/ML/predictions/predicted_images_{model_name}_{dataset_name}/'
if test_on_tiles: save_path_image_predictions = save_path_image_predictions[:-1]+'_tiles/'

In [12]:
""" Load model """
ml = ML_utils(user='mikaellv')
model = keras.models.load_model(path_to_model, custom_objects={'call':CustomLoss.call})

""" Define test image paths """
path_to_images = "data/datasets/" + dataset_name + "/test/images"
path_to_masks = "data/datasets/" + dataset_name + "/test/masks"

""" Define classes """
classes = {
    0: 'not_water',
    1: 'water'
}

In [13]:
if test_on_tiles:
    """ Test on 256x256 tiles """
    test_tile_paths = [path_to_images+'/'+f for f in os.listdir(path_to_images) if not f.endswith('.tif') and not f.startswith('.')]
    images = []
    predictions = []
    if masks: segmentations = []
    # Predict tiles
    for tiles in test_tile_paths:
        sorted_tile_paths = fetch_sorted_tiles(tiles)
        preds = np.empty(sorted_tile_paths.shape, dtype=object)
        for x in range(sorted_tile_paths.shape[0]):
            for y in range(sorted_tile_paths.shape[1]):
                img = ml.LoadImage(
                    file=sorted_tile_paths[x][y], 
                    image_path=tiles,
                    mask_path=None, 
                    fetch_mask=False)
                img = np.expand_dims(img,axis=0)
                preds[x,y] = model.predict(img).squeeze()
        # Concatenate tiles to form full image
        x_dim = []
        for x in range(sorted_tile_paths.shape[0]):
            y_dim = []
            for y in range(sorted_tile_paths.shape[1]):
                y_dim.append(preds[x,y])
            x_dim.append(np.concatenate(y_dim,axis=0))
        prediction = np.concatenate(x_dim, axis=1)
        # Append predictions, images, masks to lists for visualization/performance metrics
        img_name = f'{os.path.split(tiles)[1]}.tif'
        img_shape = prediction.shape
        if masks:
            image, mask = ml.LoadImage(img_name,path_to_images,path_to_masks,fetch_mask=True)
        if not masks: image = ml.LoadImage(img_name,path_to_images,path_to_masks, fetch_mask=False)
        images.append(image[0:img_shape[0],0:img_shape[1],:])
        predictions.append(prediction)
        if masks:
            mask = ml.bin_image(mask)
            mask = ml.getSegmentationArr(mask,ml.N_CLASSES)
            segmentations.append(mask[0:img_shape[0],0:img_shape[1],:])

else:
    """ Test on full-sized images """
    n_images = len([img for img in os.listdir(path_to_images) if img.endswith('.tif')])
    test_data_generator = ml.DataGenerator(path_to_images,path_to_masks,train=False,masks=masks)
    n_classes = len(classes)
    predict_batch = 32
    n_batch = n_images // 32
    if not n_images % 32 == 0: n_batch += 1
    images = []
    predictions = []
    if masks: segmentations = []
    # Predict and append
    for batch in range(0,n_batch):
        if masks: imgs, masks = next(test_data_generator)
        else: imgs = next(test_data_generator)
        for i in range(0, len(imgs)):
            img = np.expand_dims(imgs[i],axis=0)
            images.append(imgs[i])
            prediction = model.predict(img).squeeze()
            predictions.append(prediction)
            if masks: segmentations.append(masks[i][0:prediction.shape[0],0:prediction.shape[1],:])


In [14]:
""" Set predictions to maximum value (single-band) """
for i in range(len(predictions)):
    predictions[i] = np.argmax(predictions[i],axis=-1)
    if masks: segmentations[i] = np.argmax(segmentations[i],axis=-1)

In [15]:
import cv2
""" Save predicted images for visualization """
if not os.path.exists(save_path_image_predictions):
    os.makedirs(save_path_image_predictions)
result_path = os.path.split(save_path_image_predictions)[0] + '/results/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

count = 0
if masks:
    for img, pred, mask in zip(images,predictions,segmentations):
        pred = np.rollaxis(pred,0,2)
        pred = cv2.flip(pred,1)
        pred = cv2.rotate(pred,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        fig, axs = plt.subplots(1,3,figsize=(25,25))
        plt.tight_layout()

        axs[0].imshow(img[:,:,0],cmap='PuBuGn_r')
        axs[1].imshow(pred)
        axs[2].imshow(mask)

        axs[0].set_title("Original Image")
        axs[1].set_title("Prediction")
        axs[2].set_title("Ground Truth")

        plt.savefig(save_path_image_predictions + str(count) + '.png')
        count += 1
else:
    for img, pred in zip(images,predictions):
        pred = np.rollaxis(pred,0,2)
        pred = cv2.flip(pred,1)
        pred = cv2.rotate(pred,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        fig, axs = plt.subplots(1,2,figsize=(25,25))
        plt.tight_layout()

        axs[0].imshow(img[:,:,0],cmap='PuBuGn_r')
        axs[1].imshow(pred)
        
        axs[0].set_title("Original Image")
        axs[1].set_title("Prediction")

        plt.savefig(save_path_image_predictions + str(count) + '.png')
        count += 1

/tmp/ipykernel_3801371/686485842.py:33: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(1,2,figsize=(25,25))


In [16]:
""" Calculate performance """
if masks:
    # Flatten arrays
    print("[INFO] Flattening all numpy arrays... this may take a couple of minutes.")
    predictions = np.concatenate([np_array.ravel() for np_array in predictions])
    segmentations = np.concatenate([np_array.ravel() for np_array in segmentations])

    print("[INFO] Calculating precision/recall...")
    precision = metrics.precision_score(segmentations, predictions, average='weighted')
    recall = metrics.recall_score(segmentations, predictions, average='weighted')
    f1 = (2*precision*recall)/(recall+precision)
    global_jaccard = metrics.jaccard_score(segmentations, predictions, average='weighted')
    class_based_jaccard = metrics.jaccard_score(segmentations, predictions, average=None)
    conf_mat = metrics.confusion_matrix(segmentations, predictions)
    water_acc = (conf_mat[0,0])/(conf_mat[0,0]+conf_mat[0,1])

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"Confusion matrix: \n {conf_mat}")
    with open(result_path+"performance_metrics.csv", 'w') as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"Test Data: {dataset_name}\n")
        f.write(f"Precision: {precision}\nRecall: {recall}\nF1-Score: {f1}\nWater accuracy: {water_acc}\n")
        f.write(f"Weighted average Jaccard index: {global_jaccard}\nJaccard by class: {class_based_jaccard}\n")
        f.write(f"Confusion matrix:\n{conf_mat}")
else: print("[SKIPPING] No ground truth for this test data.")

[SKIPPING] No ground truth for this test data.
